In [1]:
import torch
import pickle
import numpy as np
import random
from scipy import spatial

In [2]:
emb = torch.load('./state/fb15k237_fed3_fed_TransE.best', map_location=torch.device('cpu'))

In [3]:
ent_embed = emb['ent_embed']

In [4]:
rel_embed = emb['rel_embed']

In [5]:
data = pickle.load(open("Fed_data/FB15K237-Fed3.pkl", "rb" ))

#### ERR

In [6]:
# assume the adversary knows all information of the client 0
c0_ent = np.unique(data[0]['train']['edge_index'])

In [7]:
# creat entity pool of selected entities (adversary knowledge) in a global view
c0_ent_embed_dict = {}
value = ent_embed[0]
for idx,ent in enumerate(c0_ent):
    c0_ent_embed_dict[ent] = value[idx]

In [8]:
# map local to global
c0_mapping = dict(zip(data[0]['train']['edge_index'][0], data[0]['train']['edge_index_ori'][0]))
c0_mapping.update(dict(zip(data[0]['train']['edge_index'][1], data[0]['train']['edge_index_ori'][1])))

c0_ent_embed_dict_mapped = dict((c0_mapping[key], value) for (key, value) in c0_ent_embed_dict.items())
c0_ent_pool_mapped = [c0_mapping[i] for i in c0_ent]

##### Client 1

In [27]:
# map local to global
c1_mapping = dict(zip(data[1]['train']['edge_index'][0], data[1]['train']['edge_index_ori'][0]))
c1_mapping.update(dict(zip(data[1]['train']['edge_index'][1], data[1]['train']['edge_index_ori'][1])))

In [28]:
c1_ent = np.unique(data[1]['train']['edge_index'])

random.seed(10)
np.random.seed(10)
p = 1

c1_ent_pool = np.random.choice(c1_ent, int(p * len(c1_ent)), replace = False)
c1_ent_embed = ent_embed[1][[c1_ent_pool]]

c1_ent_pool_mapped = [c1_mapping[i] for i in c1_ent_pool]

In [29]:
syn_ent_list = [] # synthetic entity label

for i in c1_ent_pool:
    c1_ent_embed = ent_embed[1][i]
    count = 0
    loss_bound = 0
    ent_idx = []
    for j in c0_ent_embed_dict_mapped:
        loss = spatial.distance.cosine(c1_ent_embed.detach().numpy(), c0_ent_embed_dict_mapped[j].detach().numpy())
        if count == 0: # first round
            loss_bound = loss
            ent_idx.append(j)
            count += 1
        else:
            if loss < loss_bound:
                loss_bound = loss
                ent_idx.append(j)
    syn_ent_list.append(ent_idx[-1]) # global index of the entity

In [30]:
tru_ent_list = [c1_mapping[i] for i in c1_ent_pool]

In [31]:
# calculate the number of correct reconstruction
sum(first == second for (first, second) in zip(syn_ent_list, tru_ent_list)) / len(c1_ent)

0.014542589010674118

#### TRR

In [32]:
c0_rel = np.unique(data[0]['train']['edge_type_ori'])
# creat relation pool based on selected entities (global relation index)
c1_triple_all = np.array([data[1]['train']['edge_index_ori'][0], 
                          data[1]['train']['edge_type_ori'], 
                          data[1]['train']['edge_index_ori'][1]])

In [33]:
tru_trr_list = []
# the adversary knows all relation embeddings and their corresponding index, so here we use ori directly
len_c1_triple = c1_triple_all[0].shape[0]
for i in range(len_c1_triple):
    triple = c1_triple_all[:,i]
    h, r, t= triple[0], triple[1], triple[2]
    if (h in c1_ent_pool_mapped) and (t in c1_ent_pool_mapped):
        if h not in tru_trr_list:
            tru_trr_list.append(h)
        if t not in tru_trr_list:
            tru_trr_list.append(t)

In [34]:
syn_trr_list = []
for (first, second) in zip(syn_ent_list, tru_ent_list):
    if first == second:
        syn_trr_list.append(first)

# calculate the number of correct reconstruction
len(list(set(syn_trr_list).intersection(tru_trr_list))) / len_c1_triple

0.0024547444284556876

##### Client 2

In [48]:
# ERR
# map local to global
c2_mapping = dict(zip(data[2]['train']['edge_index'][0], data[2]['train']['edge_index_ori'][0]))
c2_mapping.update(dict(zip(data[2]['train']['edge_index'][1], data[2]['train']['edge_index_ori'][1])))

In [49]:
c2_ent = np.unique(data[2]['train']['edge_index'])

random.seed(10)
np.random.seed(10)
p = 1

c2_ent_pool = np.random.choice(c2_ent, int(p * len(c2_ent)), replace = False)
c2_ent_embed = ent_embed[2][[c2_ent_pool]]

c2_ent_pool_mapped = [c2_mapping[i] for i in c2_ent_pool]

In [50]:
syn_ent_list = [] # synthetic entity label

for i in c2_ent_pool:
    c2_ent_embed = ent_embed[2][i]
    count = 0
    loss_bound = 0
    ent_idx = []
    for j in c0_ent_embed_dict_mapped:
        loss = spatial.distance.cosine(c2_ent_embed.detach().numpy(), c0_ent_embed_dict_mapped[j].detach().numpy())
        if count == 0: # first round
            loss_bound = loss
            ent_idx.append(j)
            count += 1
        else:
            if loss < loss_bound:
                loss_bound = loss
                ent_idx.append(j)
    syn_ent_list.append(ent_idx[-1]) # global index of the entity

In [51]:
tru_ent_list = [c2_mapping[i] for i in c2_ent_pool]
# calculate the number of correct reconstruction
sum(first == second for (first, second) in zip(syn_ent_list, tru_ent_list)) / len(c2_ent)

0.012976827094474154

In [52]:
# TRR
c0_rel = np.unique(data[0]['train']['edge_type_ori'])
# creat relation pool based on selected entities (global relation index)
c2_triple_all = np.array([data[2]['train']['edge_index_ori'][0], 
                          data[2]['train']['edge_type_ori'], 
                          data[2]['train']['edge_index_ori'][1]])

In [53]:
tru_trr_list = []
# the adversary knows all relation embeddings and their corresponding index, so here we use ori directly
len_c2_triple = c2_triple_all[0].shape[0]
for i in range(len_c2_triple):
    triple = c2_triple_all[:,i]
    h, r, t= triple[0], triple[1], triple[2]
    if (h in c2_ent_pool_mapped) and (t in c2_ent_pool_mapped):
        if h not in tru_trr_list:
            tru_trr_list.append(h)
        if t not in tru_trr_list:
            tru_trr_list.append(t)

In [54]:
syn_trr_list = []
for (first, second) in zip(syn_ent_list, tru_ent_list):
    if first == second:
        syn_trr_list.append(first)

# calculate the number of correct reconstruction
len(list(set(syn_trr_list).intersection(tru_trr_list))) / len_c2_triple

0.0022008053496499268

In [29]:
# !usr/bin env python
# -*- coding: utf-8 -*-
 
 
import re
import math
 
 
def ConvertELogStrToValue(eLogStr):
    """
    convert string of natural logarithm base of E to value
    return (convertOK, convertedValue)
    eg:
    input:  -1.1694737e-03
    output: -0.001169
    input:  8.9455025e-04
    output: 0.000895
    """
 
    (convertOK, convertedValue) = (False, 0.0)
    foundEPower = re.search("(?P<coefficientPart>-?\d+\.\d+)e(?P<ePowerPart>-\d+)", eLogStr, re.I)
    #print "foundEPower=",foundEPower
    if(foundEPower):
        coefficientPart = foundEPower.group("coefficientPart")
        ePowerPart = foundEPower.group("ePowerPart")
        #print "coefficientPart=%s,ePower=%s"%(coefficientPart, ePower)
        coefficientValue = float(coefficientPart)
        ePowerValue = float(ePowerPart)
        #print "coefficientValue=%f,ePowerValue=%f"%(coefficientValue, ePowerValue)
        #math.e= 2.71828182846
        # wholeOrigValue = coefficientValue * math.pow(math.e, ePowerValue)
        wholeOrigValue = coefficientValue * math.pow(10, ePowerValue)
 
        #print "wholeOrigValue=",wholeOrigValue;
 
        (convertOK, convertedValue) = (True, wholeOrigValue)
    else:
        (convertOK, convertedValue) = (False, 0.0)
 
    return (convertOK, convertedValue)
 
def parseIntEValue(intEValuesStr):
    # print "intEValuesStr=", intEValuesStr
    intEStrList = re.findall("-?\d+\.\d+e-\d+", intEValuesStr)
    # intEStrList = intEValuesStr.split(' ')
    # print "intEStrList=", intEStrList
    for eachIntEStr in intEStrList:
        # intValue = int(eachIntEStr)
        # print "intValue=",intValue
        (convertOK, convertedValue) = ConvertELogStrToValue(eachIntEStr)
        #print "convertOK=%s,convertedValue=%f"%(convertOK, convertedValue)
        print("eachIntEStr=%s,\tconvertedValue=%f" % (eachIntEStr, convertedValue)

In [31]:
parseIntEValue("7.036802476954472e-05")

eachIntEStr=7.036802476954472e-05,	convertedValue=0.000070
